In [3]:
# source: https://github.com/JustGlowing/minisom

from math import sqrt
import numpy as np
np.set_printoptions(suppress=True, precision=5)
import pandas as pd
pd.options.display.float_format = '{:.5f}'.format
from numpy import (array, unravel_index, nditer, linalg, random, subtract, max,
                   power, exp, pi, zeros, ones, arange, outer, meshgrid, dot,
                   logical_and, mean, std, cov, argsort, linspace, transpose,
                   einsum, prod, nan, sqrt, hstack, diff, argmin, multiply)
from numpy import sum as npsum
from numpy.linalg import norm
from collections import defaultdict, Counter
from warnings import warn
from sys import stdout
from time import time
from datetime import timedelta
import pickle
import os
from itertools import product

# for testing
from numpy.testing import assert_almost_equal, assert_array_almost_equal
from numpy.testing import assert_array_equal
import unittest

from IPython.display import display
import matplotlib.pyplot as plt
from PIL import Image

from sklearn.metrics import ndcg_score, dcg_score

In [18]:
#### Plain SOM ####

def _build_iteration_indexes(data_len, num_iterations,
                             verbose=False, random_generator=None):
    """Returns an iterable with the indexes of the samples
    to pick at each iteration of the training.
    If random_generator is not None, it must be an instalce
    of numpy.random.RandomState and it will be used
    to randomize the order of the samples."""
    iterations = arange(num_iterations) % data_len
    if random_generator:
        random_generator.shuffle(iterations)
    if verbose:
        return _wrap_index__in_verbose(iterations)
    else:
        return iterations

def _wrap_index__in_verbose(iterations):
    """Yields the values in iterations printing the status on the stdout."""
    m = len(iterations)
    digits = len(str(m))
    progress = '\r [ {s:{d}} / {m} ] {s:3.0f}% - ? it/s'
    progress = progress.format(m=m, d=digits, s=0)
    stdout.write(progress)
    beginning = time()
    stdout.write(progress)
    for i, it in enumerate(iterations):
        yield it
        sec_left = ((m-i+1) * (time() - beginning)) / (i+1)
        time_left = str(timedelta(seconds=sec_left))[:7]
        progress = '\r [ {i:{d}} / {m} ]'.format(i=i+1, d=digits, m=m)
        progress += ' {p:3.0f}%'.format(p=100*(i+1)/m)
        progress += ' - {time_left} left '.format(time_left=time_left)
        stdout.write(progress)

def fast_norm(x):
    """Returns norm-2 of a 1-D numpy array.
    * faster than linalg.norm in case of 1-D arrays (numpy 1.9.2rc1).
    """
    return sqrt(dot(x, x.T))

def asymptotic_decay(learning_rate, t, max_iter):
    """Decay function of the learning process.
    Parameters
    ----------
    learning_rate : float
        current learning rate.
    t : int
        current iteration.
    max_iter : int
        maximum number of iterations for the training.
    """
    return learning_rate / (1+t/(max_iter/2))

class plain_SOM(object):
    def __init__(self, x, y, input_len, data, sigma=1.0, learning_rate=0.5,
                 decay_function=asymptotic_decay,
                 neighborhood_function='gaussian', topology='rectangular',
                 activation_distance='euclidean', random_seed=None):
        """Initializes a Self Organizing Maps.
        A rule of thumb to set the size of the grid for a dimensionality
        reduction task is that it should contain 5*sqrt(N) neurons
        where N is the number of samples in the dataset to analyze.
        E.g. if your dataset has 150 samples, 5*sqrt(150) = 61.23
        hence a map 8-by-8 should perform well.
        Parameters
        ----------
        x : int
            x dimension of the SOM.
        y : int
            y dimension of the SOM.
        input_len : int
            Number of the elements of the vectors in input.
        sigma : float, optional (default=1.0)
            Spread of the neighborhood function, needs to be adequate
            to the dimensions of the map.
            (at the iteration t we have sigma(t) = sigma / (1 + t/T)
            where T is #num_iteration/2)
        learning_rate : initial learning rate
            (at the iteration t we have
            learning_rate(t) = learning_rate / (1 + t/T)
            where T is #num_iteration/2)
        decay_function : function (default=None)
            Function that reduces learning_rate and sigma at each iteration
            the default function is:
                        learning_rate / (1+t/(max_iterarations/2))
            A custom decay function will need to to take in input
            three parameters in the following order:
            1. learning rate
            2. current iteration
            3. maximum number of iterations allowed
            Note that if a lambda function is used to define the decay
            MiniSom will not be pickable anymore.
        neighborhood_function : string, optional (default='gaussian')
            Function that weights the neighborhood of a position in the map.
            Possible values: 'gaussian', 'mexican_hat', 'bubble', 'triangle'
        topology : string, optional (default='rectangular')
            Topology of the map.
            Possible values: 'rectangular', 'hexagonal'
        activation_distance : string, optional (default='euclidean')
            Distance used to activate the map.
            Possible values: 'euclidean', 'cosine', 'manhattan', 'chebyshev'
        random_seed : int, optional (default=None)
            Random seed to use.
        """
        if sigma >= x or sigma >= y:
            warn('Warning: sigma is too high for the dimension of the map.')

        self.som_size = x
        
        self._random_generator = random.RandomState(random_seed)

        self._learning_rate = learning_rate
        self._sigma = sigma
        
        self._input_len = input_len
        self.data = data
        
        # random initialization
        self._weights = self._random_generator.rand(x, y, self._input_len)*2-1

            
        self._weights /= linalg.norm(self._weights, axis=-1, keepdims=True)
        
        self.query_scores=np.empty(shape=(1, 20000))
        
        self._activation_map = np.zeros((x, y))
        self._neigx = arange(x)
        self._neigy = arange(y)  # used to evaluate the neighborhood function

        self.topology = topology
        self._xx, self._yy = meshgrid(self._neigx, self._neigy)
        self._xx = self._xx.astype(float)
        self._yy = self._yy.astype(float)

        self._decay_function = decay_function

        neig_functions = {'gaussian': self._gaussian}

        self.neighborhood = neig_functions[neighborhood_function]

        distance_functions = {'euclidean': self._euclidean_distance}

        self._activation_distance = distance_functions[activation_distance]
    
        # empty dataframe with column names
        self.selected_images = pd.DataFrame(columns = ['filename', 'ID', 'BMU_x', 'BMU_y'])
        self.selected_vectors = np.zeros((self.som_size, self.som_size, 128))
    
    def _check_input_len(self):
        """Checks that the data in input is of the correct shape."""
        data_len = len(self.data[0])
        if self._input_len != data_len:
            msg = 'Received %d features, expected %d.' % (data_len, self._input_len)
            raise ValueError(msg)
    
    def get_weights(self):
        """Returns the weights of the neural network."""
        return self._weights

    def _activate(self, x):
        """Updates matrix activation_map, in this matrix
           the element i,j is the response of the neuron i,j to x."""   
            
        self._activation_map = self._activation_distance(x, self._weights)

    def activate(self, x):
        """Returns the activation map to x."""
        self._activate(x)
        return self._activation_map

    def _gaussian(self, c, sigma):
        """Returns a Gaussian centered in c."""
        d = 2*sigma*sigma
        ax = exp(-power(self._xx-self._xx.T[c], 2)/d)
        ay = exp(-power(self._yy-self._yy.T[c], 2)/d)
        return (ax * ay).T  # the external product gives a matrix

    def _euclidean_distance(self, x, w):
        return linalg.norm(subtract(x, w), axis=-1)

    def winner(self, x):
        """Computes the coordinates of the winning neuron for the sample x."""
        
        self._activate(x)
        
        # index of bmu
        return unravel_index(self._activation_map.argmin(),
                             self._activation_map.shape)

    def update(self, x, win, t, max_iteration):
        """Updates the weights of the neurons.
        Parameters
        ----------
        x : np.array
            Current pattern to learn.
        win : tuple
            Position of the winning neuron for x (array or tuple).
        t : int
            Iteration index
        max_iteration : int
            Maximum number of training itarations.
        """
        eta = self._decay_function(self._learning_rate, t, max_iteration)
        # sigma and learning rate decrease with the same rule
        sig = self._decay_function(self._sigma, t, max_iteration)
        # improves the performances
        g = self.neighborhood(win, sig)*eta
        # w_new = eta * neighborhood_function * (x-w)
        
        self._weights += einsum('ij, ijk->ijk', g, x-self._weights)

    def quantization(self):
        """Assigns a code book (weights vector of the winning neuron)
        to each sample in data."""
        self._check_input_len()
        winners_coords = argmin(self._distance_from_weights(), axis=1)
        return self._weights[unravel_index(winners_coords,
                                           self._weights.shape[:2])]
    
    def _distance_from_weights(self):
        """Returns a matrix d where d[i,j] is the euclidean distance between
        data[i] and the j-th weight.
        """
        input_data = array(self.data)
        weights_flat = self._weights.reshape(-1, self._weights.shape[2])
        input_data_sq = power(input_data, 2).sum(axis=1, keepdims=True)
        weights_flat_sq = power(weights_flat, 2).sum(axis=1, keepdims=True)
        cross_term = dot(input_data, weights_flat.T)
        return sqrt(-2 * cross_term + input_data_sq + weights_flat_sq.T)

    def random_weights_init(self):
        """Initializes the weights of the SOM
        picking random samples from data."""
        self._check_input_len()
        it = nditer(self._activation_map, flags=['multi_index'])
        while not it.finished:
            rand_i = self._random_generator.randint(len(self.data))
            self._weights[it.multi_index] = self.data[rand_i]
            it.iternext()

    def train(self, num_iteration, random_order=False, verbose=False):
        """Trains the SOM.
        Parameters
        ----------
        data : np.array or list
            Data matrix.
        num_iteration : int
            Maximum number of iterations (one iteration per sample).
        random_order : bool (default=False)
            If True, samples are picked in random order.
            Otherwise the samples are picked sequentially.
        verbose : bool (default=False)
            If True the status of the training
            will be printed at each iteration.
        """
        self._check_input_len()
        
        random_generator = None
        # if random_order:
        #    random_generator = self._random_generator

        iterations = _build_iteration_indexes(len(self.data), num_iteration,
                                              verbose, random_generator)

        for t, iteration in enumerate(iterations):
            self.update(self.data[iteration], self.winner(self.data[iteration]),
                        t, num_iteration)
        
        if verbose:
            print('\n quantization error:', self.quantization_error())

    def train_batch(self, num_iteration, verbose=False):
        """Trains the SOM using all the vectors in data sequentially.
        Parameters
        ----------
        data : np.array or list
            Data matrix.
        num_iteration : int
            Maximum number of iterations (one iteration per sample).
        verbose : bool (default=False)
            If True the status of the training
            will be printed at each iteration.
        """
        self.train(num_iteration, random_order=False, verbose=verbose)

    def distance_map(self):
        """Returns the distance map of the weights.
        Each cell is the normalised sum of the distances between
        a neuron and its neighbours. Note that this method uses
        the euclidean distance."""
        um = zeros((self._weights.shape[0],
                    self._weights.shape[1],
                    8))  # 2 spots more for hexagonal topology

        ii = [[0, -1, -1, -1, 0, 1, 1, 1]]*2
        jj = [[-1, -1, 0, 1, 1, 1, 0, -1]]*2

        for x in range(self._weights.shape[0]):
            for y in range(self._weights.shape[1]):
                w_2 = self._weights[x, y]
                e = y % 2 == 0   # only used on hexagonal topology
                for k, (i, j) in enumerate(zip(ii[e], jj[e])):
                    if (x+i >= 0 and x+i < self._weights.shape[0] and
                            y+j >= 0 and y+j < self._weights.shape[1]):
                        w_1 = self._weights[x+i, y+j]
                        um[x, y, k] = fast_norm(w_2-w_1)

        um = um.sum(axis=2)
        return um/um.max()

    def activation_response(self):
        """
            Returns a matrix where the element i,j is the number of times
            that the neuron i,j have been winner.
        """
        self._check_input_len()
        a = zeros((self._weights.shape[0], self._weights.shape[1]))
        for x in self.data:
            a[self.winner(x)] += 1
        return a

    def quantization_error(self):
        """Returns the quantization error computed as the average
        distance between each input sample and its best matching unit."""
        self._check_input_len()
        return norm(self.data - self.quantization(), axis=1).mean()

    def win_map(self, return_indices=False):
        """Returns a dictionary wm where wm[(i,j)] is a list with:
        - all the patterns that have been mapped to the position (i,j),
          if return_indices=False (default)
        - all indices of the elements that have been mapped to the
          position (i,j) if return_indices=True"""
        self._check_input_len()
        winmap = defaultdict(list)
        for i, x in enumerate(self.data):
            winmap[self.winner(x)].append(i if return_indices else x)
        return winmap

    def labels_map(self, labels):
        """Returns a dictionary wm where wm[(i,j)] is a dictionary
        that contains the number of samples from a given label
        that have been mapped in position i,j.
        Parameters
        ----------
        data : np.array or list
            Data matrix.
        label : np.array or list
            Labels for each sample in data.
        """
        self._check_input_len()
        if not len(self.data) == len(labels):
            raise ValueError('data and labels must have the same length.')
        winmap = defaultdict(list)
        for x, l in zip(self.data, labels):
            winmap[self.winner(x)].append(l)
        for position in winmap:
            winmap[position] = Counter(winmap[position])
        return winmap
    
    ## custom functions for displaying images ##
    
    def display_results(self):
        
        #start_time = time()
        # putting the 3 functions below together
        filepaths = pd.read_csv(r"C:\Users\KWP\bachelor_thesis\native-queries\frame-ID-to-filepath.csv",sep=' ',names=['filename', 'ID'])
        df = self.node2image(filepaths)
        self.select_images(df)
        self.display_images()

        #print("Time elapsed: %s seconds" % (time() - start_time))
        
    def node2image(self, filepaths):

        # run find_bmu to obtain BMU for each feature vector

        bmu_list = []
        for vec in self.data:
            bmu_list.append(np.asarray(self.winner(vec)))

        # storing the result in a dataframe. 
        # i-th row of the dataframe (ex: 4 1) is associated with the BMU coordinates of the i-th feature vector  
        data_ = pd.DataFrame(bmu_list, columns=['BMU_x','BMU_y'])

        # merge the 2 dataframes
        df = pd.concat([filepaths, data_], axis=1, join='inner')

        # now we have:

        # filename ID BMU_x BMU_y
        # ........ rows ........

        return df

    def select_images(self, df):

        # empty dataframe with column names
        selected_images = self.selected_images

        for i in range(self.som_size):
            for j in range(self.som_size):

                tmp = df.loc[(df.BMU_x == i) & (df.BMU_y == j)]

                if(len(tmp) != 0):

                    ## random selection (for now) ##
                    sample = tmp.sample()
                    selected_images = selected_images.append(sample)

        #print("The following images were selected: ")
        #display(selected_images)

        self.selected_images = selected_images
        
    def display_images(self):
        
        my_dir = "C:/Users/KWP/bachelor_thesis/native-queries/thumbs/"

        #fig, ax = plt.subplots(self.som_size, self.som_size, sharex='col', sharey='row', figsize=(320,180))

        for i in range(len(self.selected_images)):

            row = self.selected_images.iloc[i,:]
            x = row.BMU_x
            y = row.BMU_y
            id = row.ID
        
            # extracting selected vectors used for output - for evaluation purposes later
            self.selected_vectors[x, y] = self.data[id]
        
            #img = Image.open(my_dir + row.filename)
            #img = img.resize((320, 180))
            #img = np.asarray(img)

            #ax[x, y].imshow(img)

        #plt.tight_layout(pad=0.1, w_pad=0.1, h_pad=0.1)
        #plt.show()
        
    ## evaluation metrics ## 
    # comparing the distance of selected images on the output screen and in the feature space
    
    def mean_distance(self):
    
        # mean distance matrix[i,j] contains the mean distance of neighbors from selected_vectors[i, j]
        mean_distance_matrix = np.zeros((self.som_size, self.som_size))
        
        distance_sum = 0
        
        for i in range(self.som_size):
            for j in range(self.som_size):
                
                for index in list(direct_neighbours((i,j), self.som_size)):
                    
                    distance_sum += self._euclidean_distance(self.selected_vectors[index], self.selected_vectors[i, j])
                    
                distance_sum /= len(list(direct_neighbours((i,j), self.som_size)))
                
                mean_distance_matrix[i, j] = distance_sum
        
        return mean_distance_matrix
                
    def mean_distance_nextdoor(self):
        
        mean_distance_matrix = np.zeros((self.som_size, self.som_size))
        
        for i in range(self.som_size):
            for j in range(self.som_size):
                
                if j == 0:
                    mean_distance_matrix[i, j] = self._euclidean_distance(self.selected_vectors[i,j+1], self.selected_vectors[i, j])
                    
                elif j == self.som_size-1:
                    mean_distance_matrix[i, j] = self._euclidean_distance(self.selected_vectors[i, j-1], self.selected_vectors[i, j])
                
                else:
                    mean_distance_matrix[i, j] = self._euclidean_distance(self.selected_vectors[i, j-1], self.selected_vectors[i, j]) 
                    + self._euclidean_distance(self.selected_vectors[i, j+1], self.selected_vectors[i, j])
                    mean_distance_matrix[i, j] /= 2
                    
        return mean_distance_matrix
        
    ## Rank correlation metrics ## 
    # comparing the original ranking with SOM-induced ranking
    
    def add_ranking(self, df):
        
        selected = self.selected_images
        # merge the dataframes where ID matches
        selected = selected.merge(df, how='inner', on='ID')  

        selected['SOM_rank'] = np.arange(len(selected)) + 1
        selected['QS_rank'] = selected['query_scores'].rank(ascending=False).astype(int)
    
        self.selected_images = selected
        
    def tau(self):
            
        return self.selected_images['SOM_rank'].corr(self.selected_images['QS_rank'],method='kendall')

    def nDCG(self):
        ## use original scores, not rankings. ##
    
        # Releveance scores in actual order
        actual = self.selected_images['query_scores']
        # in ideal order
        ideal = actual.sort_values()
        
        actual = np.asarray(actual).reshape(1,100)
        ideal = np.asarray(ideal).reshape(1, 100)

        return ndcg_score(actual, ideal) 
              
# returns indexes of direct neighbors of a given position in matrix
def direct_neighbours(cell, size):
    for c in product(*(range(n-1, n+2) for n in cell)):
        if c != cell and all(0 <= n < size for n in c):
            yield c

In [20]:
#### Biased SOM ####

# where does node selection during training occur?

class biased_SOM(object):
    def __init__(self, x, y, input_len, data, sigma=1.0, learning_rate=0.5,
                 decay_function=asymptotic_decay,
                 neighborhood_function='gaussian', topology='rectangular',
                 activation_distance='euclidean', random_seed=None, auto_var_adjustment=False, var_param=0, 
                 num_images=20000, query_scores=np.zeros(shape=(20000,))):
        """Initializes a Self Organizing Maps.
        A rule of thumb to set the size of the grid for a dimensionality
        reduction task is that it should contain 5*sqrt(N) neurons
        where N is the number of samples in the dataset to analyze.
        E.g. if your dataset has 150 samples, 5*sqrt(150) = 61.23
        hence a map 8-by-8 should perform well.
        Parameters
        ----------
        x : int
            x dimension of the SOM.
        y : int
            y dimension of the SOM.
        input_len : int
            Number of the elements of the vectors in input.
        sigma : float, optional (default=1.0)
            Spread of the neighborhood function, needs to be adequate
            to the dimensions of the map.
            (at the iteration t we have sigma(t) = sigma / (1 + t/T)
            where T is #num_iteration/2)
        learning_rate : initial learning rate
            (at the iteration t we have
            learning_rate(t) = learning_rate / (1 + t/T)
            where T is #num_iteration/2)
        decay_function : function (default=None)
            Function that reduces learning_rate and sigma at each iteration
            the default function is:
                        learning_rate / (1+t/(max_iterarations/2))
            A custom decay function will need to to take in input
            three parameters in the following order:
            1. learning rate
            2. current iteration
            3. maximum number of iterations allowed
            Note that if a lambda function is used to define the decay
            MiniSom will not be pickable anymore.
        neighborhood_function : string, optional (default='gaussian')
            Function that weights the neighborhood of a position in the map.
            Possible values: 'gaussian', 'mexican_hat', 'bubble', 'triangle'
        topology : string, optional (default='rectangular')
            Topology of the map.
            Possible values: 'rectangular', 'hexagonal'
        activation_distance : string, optional (default='euclidean')
            Distance used to activate the map.
            Possible values: 'euclidean', 'cosine', 'manhattan', 'chebyshev'
        random_seed : int, optional (default=None)
            Random seed to use.
        """
        if sigma >= x or sigma >= y:
            warn('Warning: sigma is too high for the dimension of the map.')

        self.som_size = x
        self.num_images = num_images
        
        self._random_generator = random.RandomState(random_seed)

        self._learning_rate = learning_rate
        self._sigma = sigma

        ## using top 1000 images only! ##
        #ids = pd.DataFrame(frame_ids, columns=['ID'])
        #ids = ids[0:1000]
        #self.data = data[ids.ID]
        
        self.data = data
        
        # random initialization
        if auto_var_adjustment == True or var_param > 0: 
            self._input_len = input_len + 1
            self._weights = self._random_generator.rand(x, y, self._input_len)*2-1
            self.data = np.append(self.data, np.zeros((self.num_images, 1)), axis=1)
            self.data = np.append(data, np.zeros((self.num_images, 1)), axis=1)
            
        else:
            self._input_len = input_len
            self._weights = self._random_generator.rand(x, y, self._input_len)*2-1
            self.data = data
            
        self._weights /= linalg.norm(self._weights, axis=-1, keepdims=True)
        
        self.query_scores=np.empty(shape=(1, self.num_images))

        self._activation_map = np.zeros((x, y))
        self._neigx = arange(x)
        self._neigy = arange(y)  # used to evaluate the neighborhood function

        self.topology = topology
        self._xx, self._yy = meshgrid(self._neigx, self._neigy)
        self._xx = self._xx.astype(float)
        self._yy = self._yy.astype(float)

        self._decay_function = decay_function

        neig_functions = {'gaussian': self._gaussian}

        self.neighborhood = neig_functions[neighborhood_function]

        distance_functions = {'euclidean': self._euclidean_distance}

        self._activation_distance = distance_functions[activation_distance]
    
        # empty dataframe with column names
        self.selected_images = pd.DataFrame(columns = ['filename', 'ID', 'BMU_x', 'BMU_y'])
        self.selected_vectors = np.zeros((self.som_size, self.som_size, 128))
    
    ## custom functions for adding bias to SOM initialization ## 
    
    def adjust_var(self):
        # variance of randomly initialized SOM nodes (on original features)
        sum = 0
        for i in range(self.som_size):
            for j in range(self.som_size):
                sum += self._weights[i, j].var()
        print("sum of variance over all SOM nodes' original features: {}".format(sum))
            
        # if the var difference is less than 0.1, then we adjust the query scores
        for i in range(200):
            tmp = query_scores * i
            if abs(sum - tmp.var()) < 0.05:
                print("variance of adjusted query scores: {}".format(tmp.var()))
                break
                
        self.query_scores = tmp
        
    def initial_bias(self):

        # add query scores to SOM as another feature, considering the position of the nodes

        # we will divide query_scores into N intervals of equal size (where SOM has size n x n)
        # then from each interval corresponding to each row, pick N values randomly.

        # for example, for 5 x 5 SOM: divide query_score into 5 intervals
        # top 80~100% values, 60~80%, ..., 0~20%. Nodes in top row are assigned 5 random values from the first interval, and so on.

        cutpoints = []
        interval_size = 100 / self.som_size 
        for i in range(1, self.som_size):

            cutpoint = np.quantile(self.query_scores, interval_size * i * 0.01)
            cutpoints.append(cutpoint)

            if i == 1:

                bottom_interval = self.query_scores[self.query_scores < cutpoint]
                
                # assign to bottom nodes i.e. som[size-1, 0] ... som[size-1, size-1]
                for j in range(self.som_size):
                    self._weights[self.som_size-1, j, 128] = np.random.choice(bottom_interval, size=1)[0]

            else:
                interval = self.query_scores[np.logical_and(self.query_scores > cutpoints[i-2], self.query_scores < cutpoint)]
                
                # assign to middle row nodes e.g. som[1, 0] ... som[1, size-1]
                for j in range(self.som_size):
                    self._weights[self.som_size-i, j, 128] = np.random.choice(interval, size=1)[0]

                if i == self.som_size-1:

                    top_interval = self.query_scores[self.query_scores > cutpoint]         

                    # assign to top nodes i.e. som[0, 0] ... som[0, size-1]
                    for j in range(self.som_size):

                        self._weights[0, j, 128] = np.random.choice(top_interval, size=1)[0]

    # remove the extra dimension added to the data 
    def reset_data(self):
        # return? or modify via class directly?
        self.data = np.delete(self.data, -1, axis=1)
    
    def _check_input_len(self):
        """Checks that the data in input is of the correct shape."""
        data_len = len(self.data[0])
        if self._input_len != data_len:
            msg = 'Received %d features, expected %d.' % (data_len, self._input_len)
            raise ValueError(msg)

    def _activate(self, x):
        """Updates matrix activation_map, in this matrix
           the element i,j is the response of the neuron i,j to x."""   
            
        self._activation_map = self._activation_distance(x, self._weights)

    def activate(self, x):
        """Returns the activation map to x."""
        self._activate(x)
        return self._activation_mapa

    def _gaussian(self, c, sigma):
        """Returns a Gaussian centered in c."""
        d = 2*sigma*sigma
        ax = exp(-power(self._xx-self._xx.T[c], 2)/d)
        ay = exp(-power(self._yy-self._yy.T[c], 2)/d)
        return (ax * ay).T  # the external product gives a matrix

    def _euclidean_distance(self, x, w):
        return linalg.norm(subtract(x, w), axis=-1)

    def winner(self, x):
        """Computes the coordinates of the winning neuron for the sample x."""
        self._activate(x)
        
        # index of bmu
        return unravel_index(self._activation_map.argmin(),
                             self._activation_map.shape)

    def update(self, x, win, t, max_iteration):
        """Updates the weights of the neurons.
        Parameters
        ----------
        x : np.array
            Current pattern to learn.
        win : tuple
            Position of the winning neuron for x (array or tuple).
        t : int
            Iteration index
        max_iteration : int
            Maximum number of training itarations.
        """
        eta = self._decay_function(self._learning_rate, t, max_iteration)
        # sigma and learning rate decrease with the same rule
        sig = self._decay_function(self._sigma, t, max_iteration)
        # improves the performances
        g = self.neighborhood(win, sig)*eta
        # w_new = eta * neighborhood_function * (x-w)
        self._weights += einsum('ij, ijk->ijk', g, x-self._weights)

    def quantization(self):
        """Assigns a code book (weights vector of the winning neuron)
        to each sample in data."""
        self._check_input_len()
        winners_coords = argmin(self._distance_from_weights(), axis=1)
        return self._weights[unravel_index(winners_coords,
                                           self._weights.shape[:2])]
    
    def _distance_from_weights(self):
        """Returns a matrix d where d[i,j] is the euclidean distance between
        data[i] and the j-th weight.
        """
        input_data = array(self.data)
        weights_flat = self._weights.reshape(-1, self._weights.shape[2])
        input_data_sq = power(input_data, 2).sum(axis=1, keepdims=True)
        weights_flat_sq = power(weights_flat, 2).sum(axis=1, keepdims=True)
        cross_term = dot(input_data, weights_flat.T)
        return sqrt(-2 * cross_term + input_data_sq + weights_flat_sq.T)

    def random_weights_init(self):
        """Initializes the weights of the SOM
        picking random samples from data."""
        self._check_input_len()
        it = nditer(self._activation_map, flags=['multi_index'])
        while not it.finished:
            rand_i = self._random_generator.randint(len(self.data))
            self._weights[it.multi_index] = self.data[rand_i]
            it.iternext()

    def train(self, num_iteration, random_order=False, verbose=False):
        """Trains the SOM.
        Parameters
        ----------
        data : np.array or list
            Data matrix.
        num_iteration : int
            Maximum number of iterations (one iteration per sample).
        random_order : bool (default=False)
            If True, samples are picked in random order.
            Otherwise the samples are picked sequentially.
        verbose : bool (default=False)
            If True the status of the training
            will be printed at each iteration.
        """
        self._check_input_len()
        random_generator = None
        
        iterations = _build_iteration_indexes(len(self.data), num_iteration,
                                              verbose, random_generator)

        for t, iteration in enumerate(iterations):
            self.update(self.data[iteration], self.winner(self.data[iteration]),
                        t, num_iteration)
        
        if verbose:
            print('\n quantization error:', self.quantization_error())

    def train_batch(self, num_iteration, verbose=False):
        """Trains the SOM using all the vectors in data sequentially.
        Parameters
        ----------
        data : np.array or list
            Data matrix.
        num_iteration : int
            Maximum number of iterations (one iteration per sample).
        verbose : bool (default=False)
            If True the status of the training
            will be printed at each iteration.
        """
        self.train(num_iteration, random_order=False, verbose=verbose)

    def distance_map(self):
        """Returns the distance map of the weights.
        Each cell is the normalised sum of the distances between
        a neuron and its neighbours. Note that this method uses
        the euclidean distance."""
        um = zeros((self._weights.shape[0],
                    self._weights.shape[1],
                    8))  # 2 spots more for hexagonal topology

        ii = [[0, -1, -1, -1, 0, 1, 1, 1]]*2
        jj = [[-1, -1, 0, 1, 1, 1, 0, -1]]*2

        for x in range(self._weights.shape[0]):
            for y in range(self._weights.shape[1]):
                w_2 = self._weights[x, y]
                e = y % 2 == 0   # only used on hexagonal topology
                for k, (i, j) in enumerate(zip(ii[e], jj[e])):
                    if (x+i >= 0 and x+i < self._weights.shape[0] and
                            y+j >= 0 and y+j < self._weights.shape[1]):
                        w_1 = self._weights[x+i, y+j]
                        um[x, y, k] = fast_norm(w_2-w_1)

        um = um.sum(axis=2)
        return um/um.max()

    def activation_response(self):
        """
            Returns a matrix where the element i,j is the number of times
            that the neuron i,j have been winner.
        """
        self._check_input_len()
        a = zeros((self._weights.shape[0], self._weights.shape[1]))
        for x in self.data:
            a[self.winner(x)] += 1
        return a

    def quantization_error(self):
        """Returns the quantization error computed as the average
        distance between each input sample and its best matching unit."""
        self._check_input_len()
        return norm(self.data - self.quantization(), axis=1).mean()

    def win_map(self, return_indices=False):
        """Returns a dictionary wm where wm[(i,j)] is a list with:
        - all the patterns that have been mapped to the position (i,j),
          if return_indices=False (default)
        - all indices of the elements that have been mapped to the
          position (i,j) if return_indices=True"""
        self._check_input_len()
        winmap = defaultdict(list)
        for i, x in enumerate(self.data):
            winmap[self.winner(x)].append(i if return_indices else x)
        return winmap

    def labels_map(self, labels):
        """Returns a dictionary wm where wm[(i,j)] is a dictionary
        that contains the number of samples from a given label
        that have been mapped in position i,j.
        Parameters
        ----------
        data : np.array or list
            Data matrix.
        label : np.array or list
            Labels for each sample in data.
        """
        self._check_input_len()
        if not len(self.data) == len(labels):
            raise ValueError('data and labels must have the same length.')
        winmap = defaultdict(list)
        for x, l in zip(self.data, labels):
            winmap[self.winner(x)].append(l)
        for position in winmap:
            winmap[position] = Counter(winmap[position])
        return winmap
    
    ## custom functions for displaying images ##
    
    def display_results(self, frame_id_scores):
        
        #start_time = time()

        # putting the 3 functions below together
        filepaths = pd.read_csv(r"C:\Users\KWP\bachelor_thesis\native-queries\frame-ID-to-filepath.csv",sep=' ',names=['filename', 'ID'])
        df = self.node2image(filepaths)
        
        self.select_images(df, frame_id_scores)
        self.display_images()
    
        #print("Time elapsed: %s seconds" % (time() - start_time))

        # adding ranking to selected images 
        self.add_ranking(frame_id_scores)
    
    def node2image(self, filepaths):

        # run find_bmu to obtain BMU for each feature vector

        bmu_list = []
        for vec in self.data:
            bmu_list.append(np.asarray(self.winner(vec)))

        # storing the result in a dataframe. 
        # i-th row of the dataframe (ex: 4 1) is associated with the BMU coordinates of the i-th feature vector  
        data_ = pd.DataFrame(bmu_list, columns=['BMU_x','BMU_y'])

        # merge the 2 dataframes
        df = pd.concat([filepaths, data_], axis=1, join='inner')

        # now we have:

        # filename ID BMU_x BMU_y
        # ........ rows ........

        return df

    # image selection after training
    def select_images(self, df, id_scores):

        # empty dataframe with column names
        selected_images = self.selected_images

        for i in range(self.som_size):
            for j in range(self.som_size):

                tmp = df.loc[(df.BMU_x == i) & (df.BMU_y == j)]

                if(len(tmp) != 0):
                    
                    merged = tmp.merge(id_scores, how='inner', on='ID') 
                    
                    # select image with highest score
                    highest = merged.loc[merged['query_scores'].idxmax()]
                    
                    selected_images = selected_images.append(highest)

        #print("The following images were selected: ")
        #display(selected_images)

        self.selected_images = selected_images
        
    def display_images(self):

        # this function not only displays images, but also removes the extra dimension we added to the data
        # and extracts selected vectors for each SOM node
        
        my_dir = "C:/Users/KWP/bachelor_thesis/native-queries/thumbs/"

        #fig, ax = plt.subplots(self.som_size, self.som_size, sharex='col', sharey='row', figsize=(320,180))

        # remove the extra dimension we initially added to the data
        if(self._input_len == 129):
            self.reset_data()
        for i in range(len(self.selected_images)):

            row = self.selected_images.iloc[i,:]
            x = row.BMU_x
            y = row.BMU_y
            id = row.ID
        
            # extracting selected vectors used for output - for evaluation purposes later
            self.selected_vectors[x, y] = self.data[id]
        """
            img = Image.open(my_dir + row.filename)
            img = img.resize((320, 180))
            img = np.asarray(img)

            ax[x, y].imshow(img)

        plt.tight_layout(pad=0.1, w_pad=0.1, h_pad=0.1)
        plt.show()
        """
    
    def add_ranking(self, df):
        
        selected = self.selected_images
        
        selected['SOM_rank'] = np.arange(len(selected)) + 1
        selected['QS_rank'] = selected['query_scores'].rank(ascending=False).astype(int)
    
        self.selected_images = selected
    
    ## evaluation metrics ## 
    # comparing the distance of selected images on the output screen and in the feature space
    
    def mean_distance(self):
    
        # mean distance matrix[i,j] contains the mean distance of neighbors from selected_vectors[i, j]
        mean_distance_matrix = np.zeros((self.som_size, self.som_size))
        
        distance_sum = 0
        
        for i in range(self.som_size):
            for j in range(self.som_size):
                
                for index in list(direct_neighbours((i,j), self.som_size)):
                    
                    distance_sum += self._euclidean_distance(self.selected_vectors[index], self.selected_vectors[i, j])
                    
                distance_sum /= len(list(direct_neighbours((i,j), self.som_size)))
                
                mean_distance_matrix[i, j] = distance_sum
        
        return mean_distance_matrix
                
    def mean_distance_nextdoor(self):
        
        mean_distance_matrix = np.zeros((self.som_size, self.som_size))
        
        for i in range(self.som_size):
            for j in range(self.som_size):
                
                if j == 0:
                    mean_distance_matrix[i, j] = self._euclidean_distance(self.selected_vectors[i,j+1], self.selected_vectors[i, j])
                    
                elif j == self.som_size-1:
                    mean_distance_matrix[i, j] = self._euclidean_distance(self.selected_vectors[i, j-1], self.selected_vectors[i, j])
                
                else:
                    mean_distance_matrix[i, j] = self._euclidean_distance(self.selected_vectors[i, j-1], self.selected_vectors[i, j]) 
                    + self._euclidean_distance(self.selected_vectors[i, j+1], self.selected_vectors[i, j])
                    mean_distance_matrix[i, j] /= 2
                    
        return mean_distance_matrix
        
    ## Rank correlation metrics ## 
    # comparing the original ranking with SOM-induced ranking
        
    def tau(self):
            
        return self.selected_images['SOM_rank'].corr(self.selected_images['QS_rank'], method='kendall')

    def nDCG(self):
        ## use original scores, not rankings. ##
    
        # Releveance scores in actual order
        actual = self.selected_images['query_scores']
        # in ideal order
        ideal = actual.sort_values()
        
        actual = np.asarray(actual).reshape(1,100)
        ideal = np.asarray(ideal).reshape(1, 100)

        return ndcg_score(actual, ideal)        
              
# question: where does node selection take place? how could we change it?

In [6]:
query_scores = np.fromfile("native-queries/native-query-scores.bin", dtype='f')
query_scores = query_scores.reshape(327, 20000)

frame_ids = np.fromfile("native-queries/native-query-frame-IDs.bin", dtype='i')
frame_ids = frame_ids.reshape(327, 20000)

frame_features = np.fromfile("native-queries/frame-features.bin", dtype='f')
frame_features = frame_features.reshape(20000, 128)

queries = pd.read_csv(r"C:\Users\KWP\bachelor_thesis\native-queries\native-queries.csv", sep=';', names=['frame_ID', 'query'])

In [21]:
# biased SOM pipeline

biased_som = biased_SOM(10, 10, frame_features[0].shape[0], data=frame_features, sigma=8.0, learning_rate=0.5, random_seed=1903, auto_var_adjustment=True)
biased_som.train_batch(20, verbose=True)

a = pd.DataFrame(frame_ids[0], columns=['ID'])
b = pd.DataFrame(query_scores[0], columns=['query_scores'])
ab = pd.concat([a, b], axis=1, join='inner')

biased_som.display_results(ab)


data:

[[ 0.16691  0.56416  0.32702 ...  0.00883  0.00007  0.     ]
 [ 0.1293  -0.27234  0.58125 ...  0.0286  -0.03544  0.     ]
 [-0.16001 -0.19924  0.20183 ...  0.02573  0.00251  0.     ]
 ...
 [ 0.46129 -0.17013 -0.14061 ...  0.06406 -0.007    0.     ]
 [ 0.39521  0.03581 -0.06681 ...  0.03326  0.00231  0.     ]
 [ 0.22627  0.0201  -0.07286 ...  0.07446  0.02417  0.     ]]
 [ 20 / 20 ] 100% - 0:00:00 left 
 quantization error: 0.977274976722078


In [ ]:
biased_som.selected_images

In [ ]:
biased_som.mean_distance()

In [ ]:
biased_som.mean_distance_nextdoor()

In [ ]:
print(biased_som.tau())
print(biased_som.nDCG())

In [ ]:
# Running SOM on all 327 queries

mean_dist = np.zeros(shape=(10,10))
mean_distN = np.zeros(shape=(10,10))
tau = 0
nDCG = 0

start_time = time()

for i in range(0, 327):
    biased_som = biased_SOM(10, 10, frame_features[0].shape[0], data=frame_features, sigma=8.0, learning_rate=0.5, random_seed=1903, 
               query_scores=query_scores[i], auto_var_adjustment=True)
    
    biased_som.train_batch(20)
    
    a = pd.DataFrame(frame_ids[i], columns=['ID'])
    b = pd.DataFrame(query_scores[i], columns=['query_scores'])
    ab = pd.concat([a, b], axis=1, join='inner')
    
    biased_som.display_results(ab)
    
    mean_dist += biased_som.mean_distance()
    mean_distN += biased_som.mean_distance_nextdoor()
    
    tau += biased_som.tau()
    nDCG += biased_som.nDCG()

mean_dist /= 327
mean_distN /= 327

print("mean distance between surrounding neighbors: \n{}".format(mean_dist))
print("mean distance between next door neighbors: \n{}".format(mean_distN))

tau /= 327
nDCG /= 327

print("Kendall's Tau Coefficient: {}".format(tau))
print("nDCG: {}".format(nDCG))

print("Time elapsed: %s seconds" % (time() - start_time))

In [17]:
# Plain SOM pipeline

plain_som = plain_SOM(10, 10, frame_features[0].shape[0], data=frame_features, sigma=8.0, learning_rate=0.5, random_seed=1903)
plain_som.train_batch(20, verbose=True)

a = pd.DataFrame(frame_ids[0], columns=['ID'])
b = pd.DataFrame(query_scores[0], columns=['query_scores'])
ab = pd.concat([a, b], axis=1, join='inner')

plain_som.display_results()
plain_som.add_ranking(ab)

 [ 20 / 20 ] 100% - 0:00:00 left 
 quantization error: 0.9833118713591011


In [11]:
plain_som._weights.shape

(10, 10, 128)

In [ ]:
plain_som.selected_images

In [ ]:
plain_som.mean_distance()

In [ ]:
plain_som.mean_distance_nextdoor()

In [ ]:
print(plain_som.tau())
print(plain_som.nDCG())